In [29]:
from fastapi import FastAPI, Path, Query
from fastapi.testclient import TestClient
import requests
from bs4 import BeautifulSoup
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FandomWikiAPI:
    def __init__(self, base_url="https://meg-endless-reality.fandom.com"):
        self.base_url = base_url
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }

    def _make_request(self, url):
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            logger.error(f"Ошибка запроса к {url}: {e}")
            return None

    def get_all_pages(self):
        logger.info("Получение списка всех страниц")
        response = self._make_request(f"{self.base_url}/wiki/Special:AllPages")
        if not response:
            return []

        soup = BeautifulSoup(response.text, 'html.parser')
        pages = []

        links = soup.find_all('a', href=True)
        for link in links:
            href = link['href']
            if '/wiki/' in href and 'Special:' not in href and 'User:' not in href:
                title = link.text.strip()
                if title and title not in ['Wiki', 'Main Page', 'Home']:
                    pages.append({
                        'title': title,
                        'url': href if href.startswith('http') else self.base_url + href
                    })

        logger.info(f"Найдено {len(pages)} страниц")
        return pages[:20]

    def get_page_content(self, page_url):
        logger.info(f"Получение содержимого страницы: {page_url}")
        if not page_url.startswith('http'):
            page_url = self.base_url + page_url

        response = self._make_request(page_url)
        if not response:
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find('h1')
        if not title:
            title = soup.find('title')
        title_text = title.text.strip() if title else "Неизвестно"

        content = soup.find('div', class_='mw-parser-output')
        if not content:
            content = soup.find('div', id='content')
        if not content:
            content = soup.find('main')
        if not content:
            content = soup.find('article')

        if content:
            for elem in content.find_all(['script', 'style']):
                elem.decompose()

            text_content = content.get_text(separator='\n', strip=True)
            text_content = ' '.join(text_content.split())
        else:
            text_content = "Контент не найден"

        logger.info(f"Получено содержимое страницы '{title_text}'")
        return {
            'title': title_text,
            'content': text_content,
            'url': page_url
        }

app = FastAPI()
wiki_api = FandomWikiAPI()

@app.get("/pages")
def get_pages(limit: int = Query(10, description="Количество страниц для возврата")):
    logger.info(f"Запрос на получение страниц с лимитом: {limit}")
    pages = wiki_api.get_all_pages()[:limit]
    return {
        "total": len(pages),
        "pages": pages
    }

@app.get("/pages/{page_title}/content")
def get_page_content(
    page_title: str = Path(..., description="Название страницы", alias="page_title")
):
    logger.info(f"Запрос на получение содержимого страницы: {page_title}")
    page_url = f"/wiki/{page_title}"
    content = wiki_api.get_page_content(page_url)

    if not content:
        logger.warning(f"Страница '{page_title}' не найдена")
        return {"error": "Страница не найдена"}

    return content

@app.get("/search")
def search_pages(
    query: str = Query(..., description="Поисковый запрос"),
    max_results: int = Query(5, description="Максимальное количество результатов")
):
    logger.info(f"Поиск страниц по запросу: '{query}', макс. результатов: {max_results}")
    all_pages = wiki_api.get_all_pages()

    results = [
        page for page in all_pages
        if query.lower() in page['title'].lower()
    ][:max_results]

    return {
        "query": query,
        "found": len(results),
        "results": results
    }

#Тест
client = TestClient(app)

def test_api():
    logger.info("Начало тестирования API")

    response1 = client.get("/pages?limit=3")
    logger.info(f"Тест 1 - Статус: {response1.status_code}")
    assert response1.status_code == 200
    data1 = response1.json()
    print(f"Получено страниц: {data1['total']}")

    if data1['total'] > 0:
        first_page = data1['pages'][0]['title']
        response2 = client.get(f"/pages/{first_page}/content")
        logger.info(f"Тест 2 - Статус: {response2.status_code}")
        assert response2.status_code == 200
        data2 = response2.json()
        print(f"Содержимое страницы '{data2['title']}': {data2['content'][:100]}...")

    response = client.get("/pages/entities/content")
    logger.info(f"Запрос через API - Статус: {response.status_code}")

    if response.status_code == 200:
        data = response.json()
        print(f"Заголовок: {data['title']}")
        print(f"URL: {data['url']}")
        print(f"Содержимое страницы 'Entities': {data['content']}")
        print("=" * 50)
    else:
        print(f"Ошибка: {response.json()}")

    response3 = client.get("/search?query=wiki&max_results=2")
    logger.info(f"Тест 3 - Статус: {response3.status_code}")
    assert response3.status_code == 200
    data3 = response3.json()
    print(f"Найдено результатов по запросу '{data3['query']}': {data3['found']}")

    logger.info("Тестирование завершено успешно")

if __name__ == "__main__":
    test_api()

Получено страниц: 3
Содержимое страницы 'Levels': Important Feature This content is important to game's core gameplay Reason: Levels are the main maps...
Заголовок: Entities
URL: https://meg-endless-reality.fandom.com/wiki/entities
Содержимое страницы 'Entities': Important Feature This content is important to game's core gameplay Reason: Entities are the main antagonists of the game Entities are the main antagonists of the game with most levels having them. Most entities are relatively the same while others have unique mechanics that make them difficult to survive. Common Entities [ ] Entities that are able to spawn in most levels. Humans Bacteria Skin-Stealers Clumps Hounds Stranglers Facelings Phones Memory Worms The Virus Transporters Dullers Aranea Membri Special Entities [ ] Entities that can only spawn on special levels/objectives Smilers Death Moths Twins Partygoers Partypoopers Windows The Enraged Smiler The Neighborhood Watch The Thing on Level 7 Animations Uncle Samsonite Bro